- Tensor board projection 
- Visualizing loss and network on tensorboard
- Comments 

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import mpld3
mpld3.enable_notebook()

In [3]:
from pylab import rcParams
rcParams['figure.figsize'] = 10, 10

In [4]:
import sys
import numpy as np 
import random
import math
import tensorflow as tf
import matplotlib.pyplot as plt 

sys.path.append("../Utils/")

In [5]:
from readWikiData import get_wikipedia_data

##### Get representation

In [6]:
sentences, word2idx = get_wikipedia_data(n_files=10, n_vocab=1000, by_paragraph=True)

In [7]:
def get_wiki_data_cbow(sentences, word2idx, window_size=5):
    training_data = []
    vocab_size = len(word2idx)
    for sentence in sentences:
        if len(sentence) < window_size * 2 + 1:
            continue
        for i in range(len(sentence)):
            left_context = sentence[max(i-window_size, 0): i]
            right_context = sentence[i+1:window_size + i + 1]
            centre = sentence[i]
            
            if len(left_context + right_context) < (2*window_size):
                len_left = len(left_context)
                len_right = len(right_context)
                
                if len_left < len_right:
                    right_context = sentence[i+1 : window_size + i + 1 + (len_right - len_left)]
                else:
                    left_context = sentence[max(i-window_size - (len_left - len_right), 0): i]
            
            temp = left_context + right_context
            
            if len(temp) < window_size * 2:
                print sentence
                print left_context
                print right_context
                print centre
                break 
            
            training_data.append((tuple(temp), centre))
            
            
    print training_data[:10]
    training_data = list(set(training_data))
    idx2word = {v:k for k, v in word2idx.iteritems()}
    return len(word2idx), training_data, word2idx, idx2word

In [8]:
vocab_size, training_data, word2idx, idx2word = get_wiki_data_cbow(sentences, word2idx)

[((8, 7, 225, 947, 12, 1000, 1000, 1000, 159, 15), 1000), ((1000, 7, 225, 947, 12, 1000, 1000, 1000, 159, 15), 8), ((1000, 8, 225, 947, 12, 1000, 1000, 1000, 159, 15), 7), ((1000, 8, 7, 947, 12, 1000, 1000, 1000, 159, 15), 225), ((1000, 8, 7, 225, 12, 1000, 1000, 1000, 159, 15), 947), ((1000, 8, 7, 225, 947, 1000, 1000, 1000, 159, 15), 12), ((8, 7, 225, 947, 12, 1000, 1000, 159, 15, 1000), 1000), ((7, 225, 947, 12, 1000, 1000, 159, 15, 1000, 1000), 1000), ((225, 947, 12, 1000, 1000, 159, 15, 1000, 1000, 55), 1000), ((947, 12, 1000, 1000, 1000, 15, 1000, 1000, 55, 16), 159)]


In [9]:
len(training_data)

11866966

In [10]:
training_data[:10]

[((1000, 160, 529, 1000, 1000, 1000, 4, 37, 225, 339), 1000),
 ((621, 1000, 24, 31, 1000, 2, 594, 1000, 5, 758), 6),
 ((1000, 44, 22, 1000, 1000, 718, 1000, 5, 24, 248), 17),
 ((632, 42, 1000, 8, 1000, 2, 510, 1000, 14, 2), 17),
 ((1000, 2, 1000, 1000, 1000, 6, 1000, 11, 113, 1000), 1000),
 ((1000, 5, 2, 1000, 242, 677, 25, 1000, 1000, 98), 3),
 ((1000, 1000, 61, 2, 1000, 1000, 1000, 2, 1000, 427), 4),
 ((32, 1000, 1000, 16, 45, 1000, 3, 1000, 4, 1000), 9),
 ((1000, 1000, 5, 2, 1000, 1000, 1000, 1000, 26, 395), 14),
 ((5, 43, 1000, 6, 1000, 1000, 1000, 19, 37, 1000), 560)]

##### Get batches

In [11]:
bucket_list = []

def getNextBatchCbow(bi_grams_, window_size=5, batch_size=10000):
    global bucket_list
    docs_ids_to_select = list(set(bi_grams_) - set(bucket_list))
    
    if len(docs_ids_to_select) < batch_size:
        bucket_list = []
        docs_ids_to_select = bi_grams_
        
    # Initialize two variables 
    train_X = np.ndarray(shape=(batch_size, window_size*2), dtype=np.int32)
    train_label = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    
    # Get a random set of docs 
    random_docs = random.sample(docs_ids_to_select, batch_size)
    bucket_list += random_docs
    
    
    index = 0 
    
    # Iterate threw all the docs 
    for item in random_docs:
        train_X[index] = item[0]
        train_label[index] = item[1]  
        index += 1
            
    return train_X, train_label

In [12]:
#getNextBatchCbow(training_data, 2)

##### Let's design the graph 

In [13]:
def init_weight(Mi, Mo):
    shape_sum = float(Mi + Mo) 
    return np.random.uniform(-np.sqrt(6/shape_sum),np.sqrt(6/shape_sum), [Mi, Mo])

In [14]:
embedding_size_w = 100
vocab_size = len(word2idx)
n_neg_samples = 20
learning_rate = 10e-5
epochs = 1001
batch_size=10000
mu = 0.99
window_size = 5

In [15]:
# Define placeholders for training 
train_X = tf.placeholder(tf.int32, shape=[batch_size, None])
train_label = tf.placeholder(tf.int32, shape=[batch_size, 1])

In [16]:
# Define matrix for doc_embedding and word_embedding 
W1 = tf.Variable(init_weight(vocab_size, embedding_size_w), name="W1", dtype=tf.float32)

In [17]:
# Define weights for the output unit 
W2 = tf.Variable(init_weight(vocab_size, embedding_size_w), name="W2", dtype=tf.float32)
biases = tf.Variable(tf.zeros(vocab_size))

In [18]:
print(train_X.get_shape(), train_label.get_shape(), W1.get_shape(), W2.get_shape())

(TensorShape([Dimension(10000), Dimension(None)]), TensorShape([Dimension(10000), Dimension(1)]), TensorShape([Dimension(1001), Dimension(100)]), TensorShape([Dimension(1001), Dimension(100)]))


In [19]:
embed = []

# generating a vector of size embedding_size_d
embed_w = tf.zeros([1, embedding_size_w], dtype=tf.float32)

# add all the word vecs in window_size
for j in range(window_size*2):
    embed_w += tf.nn.embedding_lookup(W1, train_X[:, j])
embed.append(embed_w)

embed = tf.concat(1, embed)/(window_size*2)

In [20]:
print(embed.get_shape())

(10000, 100)


In [21]:
loss = tf.nn.sampled_softmax_loss(weights=W2, \
                                  biases=biases, \
                                  labels=train_label, \
                                  inputs=embed, \
                                  num_sampled=n_neg_samples, \
                                  num_classes=vocab_size)

In [22]:
loss = tf.reduce_mean(loss)

In [23]:
#optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=mu).minimize(loss)
#optimizer = tf.train.AdagradOptimizer(learning_rate).minimize(loss)


global_step = tf.Variable(0, trainable=False)
starter_learning_rate = 0.01
learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           1000, 0.96, staircase=True)
# Passing global_step to minimize() will increment it at each step.
optimizer = (
    tf.train.MomentumOptimizer(learning_rate, momentum=mu).minimize(loss, global_step=global_step)
)


In [24]:
saver = tf.train.Saver()

with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    average_loss = 0
    
    for step in range(epochs):
        epoch_error = 0.0
        temp_X , temp_labels = getNextBatchCbow(window_size=5, bi_grams_=training_data)
        
        
        
        
        feed_dict = {train_X : temp_X, train_label : temp_labels}
        
        op, l = sess.run([optimizer, loss], 
                                    feed_dict=feed_dict)
        
        epoch_error += l
                
        if step % 100 == 0:
            print "Error at epoch : ", step, " = ", epoch_error
            
    save_path = saver.save(sess, "../models/model_cbow_model.ckpt")
    print("Model saved in file: %s" % save_path)

Error at epoch :  0  =  3.44129443169
Error at epoch :  100  =  2.15142416954
Error at epoch :  200  =  1.91867232323
Error at epoch :  300  =  1.88222384453
Error at epoch :  400  =  1.92942845821


KeyboardInterrupt: 

##### Embeddings 

In [ ]:
W1_embedding = None
W2_embedding = None 

with tf.Session() as sess:
    saver = tf.train.Saver()
    # Restore variables from disk.
    saver.restore(sess, "../models/model_cbow_model.ckpt")
    print("Model restored.")
    
    # Normalize word2vec 
    W1_embedding = W1.eval()
    
    # Normalize word2vec 
    W2_embedding = W2.eval()

In [ ]:
W1_embedding.shape

In [ ]:
W2_embedding.shape

In [ ]:
word2vec = np.mean([W1_embedding, W2_embedding], axis=0)

In [ ]:
word2vec.shape

##### Projection of embeddings using t-SNE 

In [ ]:
idx2word = {v:k for k, v in word2idx.items()}

In [ ]:
from sklearn.manifold import TSNE
model = TSNE()
Z = model.fit_transform(word2vec) 

In [ ]:
plt.scatter(Z[:,0], Z[:,1])
for i in xrange(len(idx2word)):
    try:
        plt.annotate(s=idx2word[i].encode("utf8"), xy=(Z[i,0], Z[i,1]))
    except:
        print "bad string:", idx2word[i]
plt.show()